# Multitasking, Multithreading, and Distributed

## Tasks, what are they?

Julia has the concept of a "task", which basically encapsulates some code that executes on the CPU. There is a "root task", which is what executes your code initially when you start running your Julia code (in the REPL or from a script), but your code (or code in other Julia libraries) can start extra tasks too. Each task can run different code, and if more than one task exists, they'll each take turns running. If you have more than one Julia task that is ready to run, then Julia will run one task at a time on a given CPU, until that task "yields" (sleeps, waits on something to happen, etc.), and then Julia will switch to running another task. This mode of operation is commonly called "multitasking" or "green threading".

To better understand how this works, let's actually spawn some tasks! There are a few ways to spawn tasks - let's use the built-in `@async` macro to create our tasks all on this same CPU (we'll get more adventurous in the next section):

In [7]:
@sync for i in 1:5
    @async for j in 1:3
        @info "Hello from task $(i)!"
    end
end

[ Info: Hello from task 1!
[ Info: Hello from task 2!
[ Info: Hello from task 3!
[ Info: Hello from task 4!
[ Info: Hello from task 5!
[ Info: Hello from task 1!
[ Info: Hello from task 2!
[ Info: Hello from task 3!
[ Info: Hello from task 4!
[ Info: Hello from task 5!
[ Info: Hello from task 1!
[ Info: Hello from task 2!
[ Info: Hello from task 3!
[ Info: Hello from task 4!
[ Info: Hello from task 5!


Above, we started 5 tasks with `@async`, and each of these tasks said hello 3 times. We see that, after task 1 says hello, then task 2 says hello, then 3, etc., and then we loop back around. This behavior illustrates that tasks can trade off executing, and so task 2 doesn't have to wait on task 1 to completely finish - task 2 can execute while task 1 still isn't finished running.

You'll notice that we put an `@sync` call on the outer loop - why do we do this? Because our tasks run asynchronously, by the time we reach the final `end`, our tasks may not have even started running yet, and Jupyter doesn't wait around for our tasks to finish before showing us what got printed within those loops. What `@sync` does is it waits for all tasks created within the block that it wraps (the outer loop) to complete before continuing on. In this case, it makes sure that all the tasks get the chance to run to completion (and print their output) before letting Jupyter send that output to our screen. It's something of a minor technical detail here, but this asynchronous behavior (and the need to "synchronize" like this) is key to multitasking in general.

This pattern we see above is good for "embarrasingly parallel" algorithms, where our tasks don't need to share data or depend on each other (they're fully independent). When our algorithms need to share data or communicate, we have tools available to help. In particular, tasks can wait on each other to complete:

In [8]:
t1 = @async println("I happen first")
t2 = @async begin
    wait(t1)
    println("I happen second")
end
wait(t2)

I happen first
I happen second


Here we see that task 2 waits on task 1 to finish before it prints, so in effect, we see task 1's println before we see task 2's println.

But furthermore, tasks also have return values (just like normal Julia functions), and we can see the return value of a task by fetching it (which also waits for the task to finish):

In [10]:
t1 = @async 1+1
t2 = @async fetch(t1)*3
fetch(t2)

6

Here, task 2 uses the result of task 1 to compute its result, so task 2 depends on task 1.

We can take this further by making a task fetch from multiple other tasks, or having a task's result be propagated to multiple other "downstream" tasks that might need it. By building up a "graph" of tasks which depend on each other (and synchronize and communicate via `wait`, `fetch`, or other mechanisms like `Channel`s), we can build very "concurrent" programs ("concurrent" really just means "many independent things running in tandem or simultaneously). This kind of "multitasking" paradigm is really powerful for building complex programs, like data analysis pipelines, web servers, spreadsheets, and much, much more. However, the model as presented is limited in how far it can scale: everything we've seen so far only runs on a single CPU, and modern computers typically have many more than one CPU available. What if we want to be able to make use of these extra CPUs?

## Gotta go fast

If you have multiple CPUs, you can tell Julia to start with multiple threads (like with `julia -t6` to start Julia with 6 threads), and then Julia can run multiple tasks at the same time, by putting a different task on each thread (which will then each run on a different CPU). This is commonly called "multithreading", and is probably the most common mechanism that users and libraries will use to accelerate algorithms. Of course, not all algorithms support multithreaded execution - algorithms which can't be split up into many separate tasks are called "sequential" or "serial". While all algorithms have some amount of sequential behavior, many algorithms can be "parallelized" by running portions of them independently. Julia's multithreading facilities make it easy to parallelize algorithms when it's possible to do so.

Let's first see what it looks like to use multithreading, with Julia's `Threads.@spawn` macro:

In [1]:
@sync for i in 1:5
    Threads.@spawn for j in 1:3
        @info "Hello from task $(i)!"
    end
end

[ Info: Hello from task 4!
[ Info: Hello from task 2!
[ Info: Hello from task 3!
[ Info: Hello from task 1!
[ Info: Hello from task 5!
[ Info: Hello from task 1!
[ Info: Hello from task 1!
[ Info: Hello from task 4!
[ Info: Hello from task 3!
[ Info: Hello from task 2!
[ Info: Hello from task 5!
[ Info: Hello from task 3!
[ Info: Hello from task 4!
[ Info: Hello from task 2!
[ Info: Hello from task 5!


This doesn't look really much different from `@async` - things are a little bit more out-of-order, but we still see each value of `i` printed 3 times. That our tasks are running on different threads can only really be seen by printing which thread our task is actively running on:

In [3]:
@sync for i in 1:5
    Threads.@spawn @info "Task $(i) running on thread $(Threads.threadid())"
end

[ Info: Task 3 running on thread 1
[ Info: Task 4 running on thread 4
[ Info: Task 2 running on thread 2
[ Info: Task 5 running on thread 3
[ Info: Task 1 running on thread 6


Now we see that our tasks are all running on different threads (which each will run on a different CPU core). Everything works otherwise the same as `@async` - tasks can wait on each other, fetch each other's results, etc., but they also happen to be able to do this while running on different threads.

## I need more Julias!

If you're lucky enough to have multiple servers at your disposal, you might be wondering if you can parallelize code across those servers. You indeed can, and it can look quite similar to what we can do with multithreading, with a few key caveats that we'll discuss soon. We can extend the idea of Julia's tasks to multiple servers by using Julia's Distributed standard library, which makes it easy to run tasks on remote servers. Let's setup some workers locally, just to see how this works:

In [ ]:
using Distributed

# Let's start up some local workers
if VERSION >= v"11-"
    # If you're using Julia >=1.11, then we can safely use multiple threads
    addprocs(;exeflags=["--project=$(pwd())", "--threads=2"])
elseif Threads.nthreads() == 1
    # Julia <1.11 cannot safely mix multiple threads and Distributed workers
    addprocs(;exeflags="--project=$(pwd())")
end

Note that we're only adding workers on our local system - this keeps it simpler for everyone at the workshop today, but you can always connect to actual remote servers with `addprocs`, and everything will work basically the same! Additionally, if you're not using Julia 1.11 or higher, there are some bugs within Distributed that make it unsafe to use when Julia is running with multiple threads, so we won't start any workers here. But you can follow along all the same!